# 산불데이터와 행정구역코드 매핑하기

### 패키지 불러오기 및 데이터 경로 설정
분석에 필요한 pandas, geopandas, os 등 주요 패키지를 import하고 데이터 경로를 지정합니다. 시도/시군구/읍면동/리 단위의 행정구역을 포함하고 있는 `shp` 파일은 GeoPandas를 이용해 읽어옵니다.

In [43]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import re

In [44]:
path = './data'
sido = gpd.read_file(os.path.join(path, '../data/ctprvn_20230729/ctprvn.shp'), encoding='cp949')
sgg = gpd.read_file(os.path.join(path, '../data/sig_20230729/sig.shp'), encoding='cp949')
emd = gpd.read_file(os.path.join(path, '../data/emd_20230729/emd.shp'), encoding='cp949')
li = gpd.read_file(os.path.join(path, '../data/li_20230729/li.shp'), encoding='cp949')

In [45]:
# 시도 데이터 확인
sido.head()

,CTPRVN_CD,CTP_ENG_NM,CTP_KOR_NM,geometry
0,11,Seoul,서울특별시,"POLYGON ((966987.226 1941110.946, 966987.119 1..."
1,26,Busan,부산광역시,"MULTIPOLYGON (((1148194.981 1685460.055, 11481..."
2,27,Daegu,대구광역시,"POLYGON ((1087859.999 1760097.461, 1087859.951..."
3,28,Incheon,인천광역시,"MULTIPOLYGON (((847834.755 1881816.658, 847840..."
4,29,Gwangju,광주광역시,"POLYGON ((932712.687 1696168.692, 932781.68 16..."


In [46]:
# 시군구 데이터 확인
sgg.head()

,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,geometry
0,11110,Jongno-gu,종로구,"POLYGON ((956615.453 1953567.199, 956621.579 1..."
1,11140,Jung-gu,중구,"POLYGON ((957890.386 1952616.746, 957909.908 1..."
2,11170,Yongsan-gu,용산구,"POLYGON ((953115.761 1950834.084, 953114.206 1..."
3,11200,Seongdong-gu,성동구,"POLYGON ((959681.109 1952649.605, 959842.412 1..."
4,11215,Gwangjin-gu,광진구,"POLYGON ((964825.058 1952633.25, 964875.565 19..."


In [47]:
# 읍면동 데이터 확인
emd.head()

,EMD_CD,EMD_ENG_NM,EMD_KOR_NM,geometry
0,11110101,Cheongun-dong,청운동,"POLYGON ((953700.022 1954605.065, 953693.871 1..."
1,11110102,Singyo-dong,신교동,"POLYGON ((953233.465 1953996.984, 953235.183 1..."
2,11110103,Gungjeong-dong,궁정동,"POLYGON ((953560.228 1954257.466, 953561.19 19..."
3,11110104,Hyoja-dong,효자동,"POLYGON ((953519.843 1953890.785, 953518.489 1..."
4,11110105,Changseong-dong,창성동,"POLYGON ((953516.123 1953734.362, 953516.526 1..."


In [48]:
# 리 데이터 확인
li.head()

,LI_CD,LI_ENG_NM,LI_KOR_NM,geometry
0,2671025021,Dongbu-ri,동부리,"POLYGON ((1156572.843 1696804.747, 1156605.133..."
1,2671025022,Gyo-ri,교리,"POLYGON ((1155731.257 1697372.017, 1155813.409..."
2,2671025023,Sincheon-ri,신천리,"POLYGON ((1157572.293 1695843.731, 1157584.155..."
3,2671025024,Jukseong-ri,죽성리,"POLYGON ((1158010.875 1696998.51, 1158017.903 ..."
4,2671025025,Seobu-ri,서부리,"POLYGON ((1155615.822 1696551.022, 1155624.394..."


In [49]:
# 분석에 필요한 컬럼만 남기기 (한글명, 행정구역 코드)
sido = sido[['CTP_KOR_NM', 'CTPRVN_CD']]
sgg = sgg[['SIG_KOR_NM', 'SIG_CD']]
emd = emd[['EMD_KOR_NM','EMD_CD']]

In [50]:
# 산림청 산불 통계 데이터 로드
fire_df = pd.read_csv(os.path.join(path, '산림청_산불상황관제시스템 산불통계데이터_20241016.csv'), encoding='cp949')

In [51]:
# 산림청 산불 통계 데이터 확인
fire_df.head()

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계
0,2024,9,29,15:41,일,2024,9,30,16:30,전북,전북,남원,산동,부절,기,작업장실화,산업현장실화,0.31
1,2024,9,10,15:55,화,2024,9,10,18:00,경남,경남,밀양,NaN,내이,기,기타(직접입력),성묘객실화(벌집소각),0.10
2,2024,9,10,14:35,화,2024,9,10,17:52,충남,충남,부여,규암,수목,기,기타(직접입력),원인미상,0.03
3,2024,9,10,14:24,화,2024,9,10,22:00,경북,경북,상주,화동,신촌,기,기타(직접입력),조사중,1.00
4,2024,9,5,13:51,목,2024,9,5,16:10,경북,경북,안동,녹전,매정,NaN,기타(직접입력),농산폐기물소각,0.05


## 행정구역코드 준비

#### 전처리용 함수 정의

불필요한 공백 제거를 위한 함수와 시도/시군구/읍면동을 다 동일한 형식(지역명만 남기고 행정구역명칭은 제거)으로 표준화시키는 함수를 정의했습니다.

In [52]:
# 공백 제거 함수
def clean_whitespace(text):
    if pd.isna(text):
        return text
    return re.sub(r'\s+', ' ', str(text)).strip()

In [53]:
# 시도명 표준화 함수
def simplify_sido(sido_name):
    if pd.isna(sido_name):
        return sido_name
    
    # 정식 명칭을 약식으로 변환하는 매핑
    sido_mapping = {
        '서울특별시': '서울', 
        '부산광역시': '부산', 
        '대구광역시': '대구',
        '인천광역시': '인천', 
        '광주광역시': '광주', 
        '대전광역시': '대전',
        '울산광역시': '울산', 
        '세종특별자치시': '세종',
        '경기도': '경기', 
        '강원특별자치도': '강원', 
        '강원도': '강원',
        '충청북도': '충북', 
        '충청남도': '충남',
        '전라북도': '전북', 
        '전라남도': '전남',
        '경상북도': '경북', 
        '경상남도': '경남',
        '제주특별자치도': '제주'
    }
    
    return sido_mapping.get(sido_name, sido_name)

In [54]:
# 시군구명 표준화 함수
def simplify_sigungu(sigungu_name):
    if pd.isna(sigungu_name):
        return sigungu_name
    
    # 접미사 제거 패턴
    patterns = [
        (r'(\S+)시\s+(\S+)구$', r'\1 \2'),  # '청주시 흥덕구' -> '청주 흥덕'
        (r'(\S+)시\s+(\S+)군$', r'\1 \2'),  # '창원시 의창군' -> '창원 의창'
        (r'(\S+)시$', r'\1'),            # '수원시' -> '수원'
        (r'(\S+)군$', r'\1'),            # '연천군' -> '연천'
        (r'(\S+)구$', r'\1')             # '강남구' -> '강남'
    ]
    
    # 패턴에 따라 접미사 제거
    for pattern, replacement in patterns:
        match = re.match(pattern, sigungu_name)
        if match:
            if pattern == r'(\S+)시\s+(\S+)구$' or pattern == r'(\S+)시\s+(\S+)군$':
                return f"{match.group(1)} {match.group(2)}"
            else:
                return match.group(1)
    
    return sigungu_name

In [55]:
# 읍면동명 표준화 함수
def simplify_emd(emd_name):
    if pd.isna(emd_name):
        return emd_name
    
    # 접미사 제거 패턴
    patterns = [
        (r'(\S+)읍$', r'\1'),  # '구성읍' -> '구성'
        (r'(\S+)면$', r'\1'),  # '화동면' -> '화동'
        (r'(\S+)동$', r'\1')   # '삼전동' -> '삼전'
    ]
    
    # 패턴에 따라 접미사 제거
    for pattern, replacement in patterns:
        match = re.match(pattern, emd_name)
        if match:
            return match.group(1)
    
    return emd_name

In [56]:
# 행정구역 데이터 공백 제거
for col in ['CTP_KOR_NM', 'SIG_KOR_NM', 'EMD_KOR_NM']:
    if col in sido.columns:
        sido[col] = sido[col].apply(clean_whitespace)
    if col in sgg.columns:
        sgg[col] = sgg[col].apply(clean_whitespace)
    if col in emd.columns:
        emd[col] = emd[col].apply(clean_whitespace)

#### 표준화된 행정구역명 활용

행정구역코드 데이터에 표준화된 행정구역명 컬럼을 추가하고, 시군구 df에는 시도 코드를, 읍면동 df에는 시도와 시군구 코드를 추가했습니다. 그 후, 시도명, 시군구명, 읍면동명, 시도코드, 시군구코드, 읍면동코드 등을 포함하는 `region_df`를 만들었습니다.

In [57]:
# 표준화된 행정구역명 컬럼 추가
sido['CTP_simple'] = sido['CTP_KOR_NM'].apply(simplify_sido)
sgg['SIG_simple'] = sgg['SIG_KOR_NM'].apply(simplify_sigungu)
emd['EMD_simple'] = emd['EMD_KOR_NM'].apply(simplify_emd)

In [58]:
# 시도, 시군구, 읍면동 코드 각각 추가
sgg['CTPRVN_CD'] = sgg['SIG_CD'].str[:2]
emd['SIG_CD'] = emd['EMD_CD'].str[:5]
emd['CTPRVN_CD'] = emd['SIG_CD'].str[:2]

In [59]:
# 시도 데이터 확인
sido.head(10)

,CTP_KOR_NM,CTPRVN_CD,CTP_simple
0,서울특별시,11,서울
1,부산광역시,26,부산
2,대구광역시,27,대구
3,인천광역시,28,인천
4,광주광역시,29,광주
5,대전광역시,30,대전
6,울산광역시,31,울산
7,세종특별자치시,36,세종
8,경기도,41,경기
9,충청북도,43,충북


In [60]:
# 시군구 데이터 확인
sgg.head(10)

,SIG_KOR_NM,SIG_CD,SIG_simple,CTPRVN_CD
0,종로구,11110,종로,11
1,중구,11140,중,11
2,용산구,11170,용산,11
3,성동구,11200,성동,11
4,광진구,11215,광진,11
5,동대문구,11230,동대문,11
6,중랑구,11260,중랑,11
7,성북구,11290,성북,11
8,강북구,11305,강북,11
9,도봉구,11320,도봉,11


In [61]:
# 읍면동 데이터 확인
emd.head(10)

,EMD_KOR_NM,EMD_CD,EMD_simple,SIG_CD,CTPRVN_CD
0,청운동,11110101,청운,11110,11
1,신교동,11110102,신교,11110,11
2,궁정동,11110103,궁정,11110,11
3,효자동,11110104,효자,11110,11
4,창성동,11110105,창성,11110,11
5,통의동,11110106,통의,11110,11
6,적선동,11110107,적선,11110,11
7,통인동,11110108,통인,11110,11
8,누상동,11110109,누상,11110,11
9,누하동,11110110,누하,11110,11


In [62]:
# 시도-시군구-읍면동, 코드번호 통합
region_list = []

for _, sido_row in sido.iterrows():
    ctprvn_cd = sido_row['CTPRVN_CD']
    ctp_simple = sido_row['CTP_simple']
    
    # 해당 시도의 시군구 필터링
    filtered_sgg = sgg[sgg['CTPRVN_CD'] == ctprvn_cd]
    
    for _, sgg_row in filtered_sgg.iterrows():
        sig_cd = sgg_row['SIG_CD']
        sig_simple = sgg_row['SIG_simple']
        
        # 해당 시군구의 읍면동 필터링
        filtered_emd = emd[emd['SIG_CD'] == sig_cd]
        
        for _, emd_row in filtered_emd.iterrows():
            emd_cd = emd_row['EMD_CD']
            emd_simple = emd_row['EMD_simple']
            
            # dictionary 생성
            region_dict = {
                '시도': ctp_simple,
                '시군구': sig_simple,
                '읍면': emd_simple,
                '시도_시군구': f"{ctp_simple} {sig_simple}",
                '시도_시군구_읍면': f"{ctp_simple} {sig_simple} {emd_simple}",
                'CTPRVN_CD': ctprvn_cd,
                'SIG_CD': sig_cd,
                'EMD_CD': emd_cd
            }
            region_list.append(region_dict)

In [63]:
# 리스트 확인
region_list[:3]

[{'시도': '서울',
  '시군구': '종로',
  '읍면': '청운',
  '시도_시군구': '서울 종로',
  '시도_시군구_읍면': '서울 종로 청운',
  'CTPRVN_CD': '11',
  'SIG_CD': '11110',
  'EMD_CD': '11110101'},
 {'시도': '서울',
  '시군구': '종로',
  '읍면': '신교',
  '시도_시군구': '서울 종로',
  '시도_시군구_읍면': '서울 종로 신교',
  'CTPRVN_CD': '11',
  'SIG_CD': '11110',
  'EMD_CD': '11110102'},
 {'시도': '서울',
  '시군구': '종로',
  '읍면': '궁정',
  '시도_시군구': '서울 종로',
  '시도_시군구_읍면': '서울 종로 궁정',
  'CTPRVN_CD': '11',
  'SIG_CD': '11110',
  'EMD_CD': '11110103'}]

In [64]:
# DataFrame 생성
region_df = pd.DataFrame(region_list)

In [65]:
# 데이터프레임 확인
region_df.head(20)

,시도,시군구,읍면,시도_시군구,시도_시군구_읍면,CTPRVN_CD,SIG_CD,EMD_CD
0,서울,종로,청운,서울 종로,서울 종로 청운,11,11110,11110101
1,서울,종로,신교,서울 종로,서울 종로 신교,11,11110,11110102
2,서울,종로,궁정,서울 종로,서울 종로 궁정,11,11110,11110103
3,서울,종로,효자,서울 종로,서울 종로 효자,11,11110,11110104
4,서울,종로,창성,서울 종로,서울 종로 창성,11,11110,11110105
5,서울,종로,통의,서울 종로,서울 종로 통의,11,11110,11110106
6,서울,종로,적선,서울 종로,서울 종로 적선,11,11110,11110107
7,서울,종로,통인,서울 종로,서울 종로 통인,11,11110,11110108
8,서울,종로,누상,서울 종로,서울 종로 누상,11,11110,11110109
9,서울,종로,누하,서울 종로,서울 종로 누하,11,11110,11110110


## 산불데이터 정제

#### 산불데이터 전처리

산불데이터에서 누락 데이터와 공백을 처리했습니다.

In [66]:
# 산불 데이터 전처리용 DataFrame 복사
fire_df_processed = fire_df.copy()

In [67]:
# 누락 데이터 처리
fire_df_processed['발생장소_시도'] = fire_df_processed['발생장소_시도'].fillna('')
fire_df_processed['발생장소_시군구'] = fire_df_processed['발생장소_시군구'].fillna('')
fire_df_processed['발생장소_읍면'] = fire_df_processed['발생장소_읍면'].fillna('')
fire_df_processed['발생장소_동리'] = fire_df_processed['발생장소_동리'].fillna('')

In [68]:
# 산불 데이터 공백 제거
for col in ['발생장소_시도', '발생장소_시군구', '발생장소_읍면', '발생장소_동리']:
    fire_df_processed[col] = fire_df_processed[col].apply(clean_whitespace)

#### 시군구 <-> 읍면 혼동 예외 처리

산불 데이터에서 시군구에 들어가야 할 값이 읍면으로 잘못 분류된 경우를 보정했습니다. 밑의 출력 예시와 같이, '고양 덕양'은 실제로 존재하는 시군구명입니다. 그러나 이를 '고양'과 '덕양'로 분리하면 각각은 `SIG_simple` 또는 `EMD_simple` 컬럼 어디에도 단독으로 존재하지 않으므로 매핑에 실패하게 됩니다. 이를 방지하기 위해, 시군구에 복합 명칭이 들어가 있는 경우를 dict으로 저장하고, 만약 시군구와 읍면에 복합 명칭이 따로 분리되어 기록되어 있다면 읍면 값을 시군구 값과 병합하여 시군구 칸에 옮긴 후 읍면 칸은 비워서, 이후 동리 값이 읍면 값을 대체할 수 있도록 했습니다.

In [69]:
# 시군구 읍면 혼동 예시
print(emd[emd['EMD_simple'] == '지축'])
print(sgg[sgg['SIG_simple'] == '고양 덕양'])
print(sgg[sgg['SIG_simple'] == '고양'])
print(emd[emd['EMD_simple'] == '덕양'])

     EMD_KOR_NM    EMD_CD EMD_simple SIG_CD CTPRVN_CD
1762        지축동  41281109         지축  41281        41
   SIG_KOR_NM SIG_CD SIG_simple CTPRVN_CD
92    고양시 덕양구  41281      고양 덕양        41
Empty DataFrame
Columns: [SIG_KOR_NM, SIG_CD, SIG_simple, CTPRVN_CD]
Index: []
Empty DataFrame
Columns: [EMD_KOR_NM, EMD_CD, EMD_simple, SIG_CD, CTPRVN_CD]
Index: []


In [70]:
# 시군구 수동 보정 dict
manual_fix_dict = {
    '성남': ['수정', '중원', '분당'],
    '고양': ['덕양', '일산동', '일산서'],
    '부천': ['오정', '원미', '소사'],
    '수원': ['장안', '권선', '팔달', '영통'],
    '안산': ['단원', '상록'],
    '안양': ['동안', '만안'],
    '용인': ['기흥', '수지', '처인'],
    '창원': ['성산', '의창', '마산합포', '진해', '마산회원'],
    '포항': ['남', '북'],
    '천안': ['동남', '서북'],
    '전주': ['완산', '덕진'],
    '청주': ['상당', '서원', '흥덕', '청원'],
}

In [71]:
# 시군구 수동 보정 함수
def manual_fix(row):
    시도 = row['발생장소_시도']
    시군구 = row['발생장소_시군구']
    읍면 = row['발생장소_읍면']

    if pd.isna(읍면):
        return row

    if 시군구 in manual_fix_dict:
        if 읍면 in manual_fix_dict[시군구]:
            row['발생장소_시군구'] = f"{시군구} {읍면}"
            row['발생장소_읍면'] = ""
    return row

fire_df_processed = fire_df_processed.apply(manual_fix, axis=1)

In [72]:
# 읍면이 없는 경우 동리 사용
def merge_emd_li(row):
    if row['발생장소_읍면']:
        return row['발생장소_읍면']
    else:
        return row['발생장소_동리']

fire_df_processed['발생장소_읍면동'] = fire_df_processed.apply(merge_emd_li, axis=1)

#### 편입 처리

산불데이터의 지역 중 군위군은 대구로 편입된 상태이기 때문에 시도를 변경해주었습니다.

In [73]:
# 군위군 --> 대구광역시 편입 처리
fire_df_processed.loc[
    (fire_df_processed['발생장소_시도'] == '경북') & 
    (fire_df_processed['발생장소_시군구'] == '군위'),
    '발생장소_시도'] = '대구'

#### 시도-시군구-읍면동 조합
산불데이터에 시도-시군구와 시도-시군구-읍면동을 조합한 칼럼을 추가했습니다.

In [74]:
# 시도-시군구-읍면동 조합 컬럼 생성
fire_df_processed['발생장소_시도_시군구'] = fire_df_processed.apply(
    lambda row: f"{row['발생장소_시도']} {row['발생장소_시군구']}" if row['발생장소_시도'] and row['발생장소_시군구'] else "", axis=1)
fire_df_processed['발생장소_시도_시군구_읍면동'] = fire_df_processed.apply(
    lambda row: f"{row['발생장소_시도']} {row['발생장소_시군구']} {row['발생장소_읍면동']}" if row['발생장소_시도'] and row['발생장소_시군구'] and row['발생장소_읍면동'] else "", axis=1)

In [75]:
fire_df_processed[['발생장소_시도', '발생장소_시도_시군구', '발생장소_시도_시군구_읍면동']].head()

,발생장소_시도,발생장소_시도_시군구,발생장소_시도_시군구_읍면동
0,전북,전북 남원,전북 남원 산동
1,경남,경남 밀양,경남 밀양 내이
2,충남,충남 부여,충남 부여 규암
3,경북,경북 상주,경북 상주 화동
4,경북,경북 안동,경북 안동 녹전


## 산불데이터에 행정구역코드 매핑

#### 수동 매핑 딕셔너리 적용

초기 실행 결과, 주소 값에 이상이 있어 매핑에 실패하는 70개의 데이터가 있다는 것을 확인하고, 이를 수동 매핑하기 위해 수작업으로 정리한 dict를 만들었습니다. 재현 가능성을 위해 코드 내 dict로 포함하였으며, 행정 코드 (`CTPRVN_CD`, `SIG_CD`, `EMD_CD`)를 직접 할당하였고, `매핑방법`에 수동 처리 원인(매핑 실패 원인)까지 함께 기록했습니다.

In [76]:
# 수동 매핑 dict
manual_dict = {
        "강원|정선|남면|유평": {
            "CTPRVN_CD": "51",
            "SIG_CD": "51770",
            "EMD_CD": "51770320",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (접미사)"
        },
        "강원|인제|북하|용대": {
            "CTPRVN_CD": "51",
            "SIG_CD": "51810",
            "EMD_CD": "51810320",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 북)"
        },
        "강원|정선|귤암|": {
            "CTPRVN_CD": "51",
            "SIG_CD": "51770",
            "EMD_CD": "51770250",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (누락 정선 정선)"
        },
        "경기|군포시||둔대": {
            "CTPRVN_CD": "41",
            "SIG_CD": "41410",
            "EMD_CD": "41410106",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (접미사)"
        },
        "경기|남양주|별내|용암": {
            "CTPRVN_CD": "41",
            "SIG_CD": "41360",
            "EMD_CD": "41360310",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (중복 존재 별내동)"
        },
        "경기|광주|남송|귀여": {
            "CTPRVN_CD": "41",
            "SIG_CD": "41610",
            "EMD_CD": "41610350",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 남종)"
        },
        "경기|화성|송산|마산": {
            "CTPRVN_CD": "41",
            "SIG_CD": "41590",
            "EMD_CD": "41590340",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (중복 존재 송산동)"
        },
        "경기|화성|송산|독지": {
            "CTPRVN_CD": "41",
            "SIG_CD": "41590",
            "EMD_CD": "41590340",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (중복 존재 송산동)"
        },
        "경기|파주|월룡|도내": {
            "CTPRVN_CD": "41",
            "SIG_CD": "41480",
            "EMD_CD": "41480310",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 월롱)"
        },
        "경기|남양주|별내|광전": {
            "CTPRVN_CD": "41",
            "SIG_CD": "41360",
            "EMD_CD": "41360310",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (중복 존재 별내동)"
        },
        "경기|남양주|별내|청학": {
            "CTPRVN_CD": "41",
            "SIG_CD": "41360",
            "EMD_CD": "41360310",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (중복 존재 별내동)"
        },
        "경기|고양|덕양구|지축": {
            "CTPRVN_CD": "41",
            "SIG_CD": "41281",
            "EMD_CD": "41281109",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (접미사)"
        },
        "경기|광주|오포|신현": {
            "CTPRVN_CD": "41",
            "SIG_CD": "41610",
            "EMD_CD": "41610115",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (폐지 신현)"
        },
        "경기|가평|청펑|대성": {
            "CTPRVN_CD": "41",
            "SIG_CD": "41820",
            "EMD_CD": "41820325",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 청평)"
        },
        "경기|연천|마산|광동": {
            "CTPRVN_CD": "41",
            "SIG_CD": "41800",
            "EMD_CD": "41800340",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 미산)"
        },
        "경기|수원 장안|장안|상광교": {
            "CTPRVN_CD": "41",
            "SIG_CD": "41111",
            "EMD_CD": "41111138",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (원본 칸 구분 오류)"
        },
        "경기|화성|송산|천동": {
            "CTPRVN_CD": "41",
            "SIG_CD": "41590",
            "EMD_CD": "41590340",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (중복 존재 송산동)"
        },
        "경기|화성|성산|고포": {
            "CTPRVN_CD": "41",
            "SIG_CD": "41590",
            "EMD_CD": "41590340",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 송산, 중복 존재 송산동)"
        },
        "경남|진해|진례|고모": {
            "CTPRVN_CD": "48",
            "SIG_CD": "48250",
            "EMD_CD": "48250330",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (누락 김해)"
        },
        "경남|의령|영덕|죽전": {
            "CTPRVN_CD": "48",
            "SIG_CD": "48720",
            "EMD_CD": "48720350",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 용덕)"
        },
        "경남|진주||주악": {
            "CTPRVN_CD": "48",
            "SIG_CD": "48170",
            "EMD_CD": "48170102",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 주약)"
        },
        "경남|산청|신동|장천": {
            "CTPRVN_CD": "48",
            "SIG_CD": "48860",
            "EMD_CD": "48860400",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 신등)"
        },
        "경남|산청|사천|내대": {
            "CTPRVN_CD": "48",
            "SIG_CD": "48860",
            "EMD_CD": "48860360",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 시천)"
        },
        "경남|창원 의창||소담": {
            "CTPRVN_CD": "48",
            "SIG_CD": "48121",
            "EMD_CD": "48121104",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 소답)"
        },
        "경남|창원 의창|북|대산": {
            "CTPRVN_CD": "48",
            "SIG_CD": "48121",
            "EMD_CD": "48121310",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (중복 존재 북동)"
        },
        "경남|함안|철원|장암": {
            "CTPRVN_CD": "48",
            "SIG_CD": "48730",
            "EMD_CD": "48730370",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 칠원)"
        },
        "경북|경주|진덕|모아": {
            "CTPRVN_CD": "47",
            "SIG_CD": "47130",
            "EMD_CD": "47130380",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오류 천북 )"
        },
        "경북|포항 북|홍해|대련": {
            "CTPRVN_CD": "47",
            "SIG_CD": "47113",
            "EMD_CD": "47113250",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 흥해)"
        },
        "경북|문경|신북|호암": {
            "CTPRVN_CD": "47",
            "SIG_CD": "47280",
            "EMD_CD": "47280340",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 산북)"
        },
        "경북|포항|구룡포|병포": {
            "CTPRVN_CD": "47",
            "SIG_CD": "47111",
            "EMD_CD": "47111250",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 포항 남)"
        },
        "경북|상주|중등|회상": {
            "CTPRVN_CD": "47",
            "SIG_CD": "47250",
            "EMD_CD": "47250310",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 중동)"
        },
        "경북|영천|고령|대의": {
            "CTPRVN_CD": "47",
            "SIG_CD": "47230",
            "EMD_CD": "47230380",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 고경)"
        },
        "경북|포항|홍해|덕장": {
            "CTPRVN_CD": "47",
            "SIG_CD": "47113",
            "EMD_CD": "47113250",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 흥해)"
        },
        "경북|포항 남||대이": {
            "CTPRVN_CD": "47",
            "SIG_CD": "47111",
            "EMD_CD": "47111116",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (명칭오류 대잠)"
        },
        "경북|경주|양북|입천": {
            "CTPRVN_CD": "47",
            "SIG_CD": "47130",
            "EMD_CD": "47130315",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (명칭변경 문무대왕)"
        },
        "대구|동||진안": {
            "CTPRVN_CD": "27",
            "SIG_CD": "27140",
            "EMD_CD": "27140133",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 진인)"
        },
        "부산|기장|장관|월평": {
            "CTPRVN_CD": "26",
            "SIG_CD": "26710",
            "EMD_CD": "26710320",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 정관, 중복존재 정관읍)"
        },
        "부산|서||서대신": {
            "CTPRVN_CD": "26",
            "SIG_CD": "26140",
            "EMD_CD": "26140104",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (불가 서대신동1/2/3가 미표기)"
        },
        "부산|영도||동상": {
            "CTPRVN_CD": "26",
            "SIG_CD": "26200",
            "EMD_CD": "26200121",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 동삼)"
        },
        "부산|영도||산제당": {
            "CTPRVN_CD": "26",
            "SIG_CD": "26200",
            "EMD_CD": "26200113",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (불가 정보 모호. 신선동2가 추정)"
        },
        "서울|강남|강남|개포": {
            "CTPRVN_CD": "11",
            "SIG_CD": "11680",
            "EMD_CD": "11680103",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (원본 칸 구분 오류)"
        },
        "세종|||미곡": {
            "CTPRVN_CD": "36",
            "SIG_CD": "36110",
            "EMD_CD": "36110380",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (누락 전동)"
        },
        "세종|||영곡": {
            "CTPRVN_CD": "36",
            "SIG_CD": "36110",
            "EMD_CD": "36110340",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (누락 금남)"
        },
        "세종|||고복": {
            "CTPRVN_CD": "36",
            "SIG_CD": "36110",
            "EMD_CD": "36110360",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (누락 연서)"
        },
        "인천|서구||왕길": {
            "CTPRVN_CD": "28",
            "SIG_CD": "28260",
            "EMD_CD": "28260120",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (접미사)"
        },
        "인천|서||검안": {
            "CTPRVN_CD": "28",
            "SIG_CD": "28260",
            "EMD_CD": "28260103",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 검암)"
        },
        "인천|중||운복": {
            "CTPRVN_CD": "28",
            "SIG_CD": "28110",
            "EMD_CD": "28110148",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 운북)"
        },
        "전남|보성|안양|수문": {
            "CTPRVN_CD": "46",
            "SIG_CD": "46800",
            "EMD_CD": "46800320",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (구분오류 장흥)"
        },
        "전남|보성|희천|천포": {
            "CTPRVN_CD": "46",
            "SIG_CD": "46780",
            "EMD_CD": "46780390",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 회천)"
        },
        "전남||송지|송호": {
            "CTPRVN_CD": "46",
            "SIG_CD": "46820",
            "EMD_CD": "46820340",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (누락 해남)"
        },
        "전남|보성|검백|온덕": {
            "CTPRVN_CD": "46",
            "SIG_CD": "46780",
            "EMD_CD": "46780330",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 겸백)"
        },
        "전남|영암|미왕|두억": {
            "CTPRVN_CD": "46",
            "SIG_CD": "46830",
            "EMD_CD": "46830390",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 미암)"
        },
        "전남|신안|측산|수": {
            "CTPRVN_CD": "46",
            "SIG_CD": "46910",
            "EMD_CD": "46910360",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 흑산)"
        },
        "전북|전주 완산||평화2가": {
            "CTPRVN_CD": "52",
            "SIG_CD": "52111",
            "EMD_CD": "52111133",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (세부 지역 정보 2가)"
        },
        "전북|전주 덕진||우아1가": {
            "CTPRVN_CD": "52",
            "SIG_CD": "52113",
            "EMD_CD": "52113113",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (세부 지역 정보 1가)"
        },
        "전북|전주 완산||효자": {
            "CTPRVN_CD": "52",
            "SIG_CD": "52111",
            "EMD_CD": "52111140",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (접미사)"
        },
        "전북|진안|상전면|주평": {
            "CTPRVN_CD": "52",
            "SIG_CD": "52720",
            "EMD_CD": "52720340",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (접미사)"
        },
        "전북||성송|무송": {
            "CTPRVN_CD": "52",
            "SIG_CD": "52790",
            "EMD_CD": "52790370",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (누락 고창)"
        },
        "전북|장수|변암|동화": {
            "CTPRVN_CD": "52",
            "SIG_CD": "52740",
            "EMD_CD": "52740320",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 번암)"
        },
        "충남|부여|새도|청포": {
            "CTPRVN_CD": "44",
            "SIG_CD": "44760",
            "EMD_CD": "44760430",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 세도)"
        },
        "충남|천안|목천|지산": {
            "CTPRVN_CD": "44",
            "SIG_CD": "44131",
            "EMD_CD": "44131250",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (누락 동남 목천)"
        },
        "충남|천안|목천|송전": {
            "CTPRVN_CD": "44",
            "SIG_CD": "44131",
            "EMD_CD": "44131250",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (누락 동남 목천)"
        },
        "충남|논산|가양곡|함적": {
            "CTPRVN_CD": "44",
            "SIG_CD": "44230",
            "EMD_CD": "44230400",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 가야곡)"
        },
        "충북|청주 상당|남성|귀래": {
            "CTPRVN_CD": "43",
            "SIG_CD": "43111",
            "EMD_CD": "43111310",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 낭성)"
        },
        "충북|청주 상당|남성|추정": {
            "CTPRVN_CD": "43",
            "SIG_CD": "43111",
            "EMD_CD": "43111310",
            "매핑수준": "수동 매핑",
            "매핑방법": "수동 (오타 낭성)"
        }
            }

#### 코드 매핑 함수
조합된 주소를 바탕으로 행정구역 코드를 매핑하는 함수를 정의했습니다.

##### 매핑 우선순위
1. 세종시 특수 처리
2. 시도-시군구-읍면 완전 매칭
3. 수동 매핑
4. 시도-시군구 매핑 후 읍면 부분 매칭
5. 시도-시군구만 매칭
6. 시도만 매칭
##### 세종 특수 처리 이유
세종특별자치시는 시도와 시군구가 모두 '세종'이기 때문에 산불 데이터에 잘못 분류되어 기록된 경우가 많았습니다. 그래서 시도가 세종일 경우, 시도와 시군구의 코드를 고정하고, 시군구 칸에 읍면 값이 잘못 들어가있을 경우에는 원래 읍면 칸에 들어가있던 값을 동리 칸으로 옮기고, 시군구 칸에 들어가있던 읍면 값을 읍면 칸으로 옮겼습니다.

In [77]:
# 산불 데이터에 행정구역 코드를 매핑하는 함수
def map_region_codes(row, region_df):
    result = {
        'CTPRVN_CD': None,
        'SIG_CD': None,
        'EMD_CD': None,
        '매핑수준': '매핑 실패',
        '매핑방법': None
    }
    
    # 세종시 특수 처리
    if row['발생장소_시도'] == '세종':
    # 시군구에 읍면이 잘못 들어간 경우 보정
        candidate_match = region_df[
            (region_df['시도'] == '세종') &
            (region_df['읍면'] == row['발생장소_시군구'])
        ]
        if len(candidate_match) == 1:
            # 시군구 --> 읍면 이동, 시군구는 세종으로 고정
            row['발생장소_읍면동'] = row['발생장소_시군구']
            row['발생장소_시군구'] = '세종'

    # 최종적으로 읍면 기준 매칭
        match = region_df[
            (region_df['시도'] == '세종') &
            (region_df['읍면'] == row['발생장소_읍면동'])
        ]
        if len(match) == 1:
            result['CTPRVN_CD'] = '36'  # 고정
            result['SIG_CD'] = '36110'  # 고정
            result['EMD_CD'] = match['EMD_CD'].values[0]
            result['매핑수준'] = '세종 매칭'
            result['매핑방법'] = '세종 시군구 무시'
            return result


    # 시도-시군구-읍면 완전 매칭
    if row['발생장소_시도_시군구_읍면동']:
        match = region_df[region_df['시도_시군구_읍면'] == row['발생장소_시도_시군구_읍면동']]
        if len(match) == 1:
            result['CTPRVN_CD'] = match['CTPRVN_CD'].values[0]
            result['SIG_CD'] = match['SIG_CD'].values[0]
            result['EMD_CD'] = match['EMD_CD'].values[0]
            result['매핑수준'] = '완전 매칭'
            result['매핑방법'] = '시도,시군구,읍면동'
            return result
    
    # 수동 매핑
    key = f"{row['발생장소_시도']}|{row['발생장소_시군구']}|{row['발생장소_읍면']}|{row['발생장소_동리']}"
    
    if key in manual_dict:
        info = manual_dict[key]
        result['CTPRVN_CD'] = info['CTPRVN_CD']
        result['SIG_CD'] = info['SIG_CD']
        result['EMD_CD'] = info['EMD_CD']
        result['매핑수준'] = info['매핑수준']
        result['매핑방법'] = info['매핑방법']
        return result


    # 시도-시군구 매칭 후 읍면 부분 매칭
    if row['발생장소_시도_시군구'] and row['발생장소_읍면동']:
        sig_matches = region_df[region_df['시도_시군구'] == row['발생장소_시도_시군구']]
        if len(sig_matches) > 0:
            # 읍면 부분 매칭 (포함 여부)
            for _, match in sig_matches.iterrows():
                if row['발생장소_읍면동'] in match['읍면'] or match['읍면'] in row['발생장소_읍면동']:
                    result['CTPRVN_CD'] = match['CTPRVN_CD']
                    result['SIG_CD'] = match['SIG_CD']
                    result['EMD_CD'] = match['EMD_CD']
                    result['매핑수준'] = '부분 매칭'
                    result['매핑방법'] = '시도,시군구 완전 / 읍면동 부분'
                    return result
    
    # 시도-시군구만 매칭
    if row['발생장소_시도_시군구']:
        sig_matches = region_df[region_df['시도_시군구'] == row['발생장소_시도_시군구']]
        if len(sig_matches) > 0:
            # 해당 시군구의 첫 번째 읍면동 코드 사용
            result['CTPRVN_CD'] = sig_matches['CTPRVN_CD'].values[0]
            result['SIG_CD'] = sig_matches['SIG_CD'].values[0]
            result['매핑수준'] = '시군구 매칭'
            result['매핑방법'] = '시도,시군구'
            return result
    
    # 시도만 매칭
    if row['발생장소_시도']:
        sido_matches = region_df[region_df['시도'] == row['발생장소_시도']]
        if len(sido_matches) > 0:
            result['CTPRVN_CD'] = sido_matches['CTPRVN_CD'].values[0]
            result['매핑수준'] = '시도 매칭'
            result['매핑방법'] = '시도'
            return result
    
    return result


#### 매핑 결과 저장

우선 매핑 결과를 리스트에 저장하고 이를 df으로 변환하여 산불데이터와 병합하였습니다. 

In [78]:
# 매핑 결과를 리스트에 저장
mapping_results = []

for _, row in fire_df_processed.iterrows():
    result = map_region_codes(row, region_df)
    mapping_results.append(result)

In [79]:
# 리스트 확인
mapping_results[:3]

[{'CTPRVN_CD': '45',
  'SIG_CD': '45190',
  'EMD_CD': '45190410',
  '매핑수준': '완전 매칭',
  '매핑방법': '시도,시군구,읍면동'},
 {'CTPRVN_CD': '48',
  'SIG_CD': '48270',
  'EMD_CD': '48270102',
  '매핑수준': '완전 매칭',
  '매핑방법': '시도,시군구,읍면동'},
 {'CTPRVN_CD': '44',
  'SIG_CD': '44760',
  'EMD_CD': '44760310',
  '매핑수준': '완전 매칭',
  '매핑방법': '시도,시군구,읍면동'}]

In [80]:
# 매핑 결과를 DataFrame으로 변환
mapping_df = pd.DataFrame(mapping_results)

In [81]:
# 데이터프레임 확인
mapping_df.head()

,CTPRVN_CD,SIG_CD,EMD_CD,매핑수준,매핑방법
0,45,45190,45190410,완전 매칭,"시도,시군구,읍면동"
1,48,48270,48270102,완전 매칭,"시도,시군구,읍면동"
2,44,44760,44760310,완전 매칭,"시도,시군구,읍면동"
3,47,47250,47250400,완전 매칭,"시도,시군구,읍면동"
4,47,47170,47170430,완전 매칭,"시도,시군구,읍면동"


In [82]:
# 산불 데이터와 매핑 결과를 병합
fire_df_final = pd.concat([fire_df_processed, mapping_df], axis=1)

In [83]:
# 데이터프레임 확인
print(fire_df_final.columns)
fire_df_final.head()

Index(['발생일시_년', '발생일시_월', '발생일시_일', '발생일시_시간', '발생일시_요일', '진화종료시간_년',
       '진화종료시간_월', '진화종료시간_일', '진화종료시간_시간', '발생장소_관서', '발생장소_시도', '발생장소_시군구',
       '발생장소_읍면', '발생장소_동리', '발생원인_구분', '발생원인_세부원인', '발생원인_기타', '피해면적_합계',
       '발생장소_읍면동', '발생장소_시도_시군구', '발생장소_시도_시군구_읍면동', 'CTPRVN_CD', 'SIG_CD',
       'EMD_CD', '매핑수준', '매핑방법'],
      dtype='object')


,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,...,발생원인_기타,피해면적_합계,발생장소_읍면동,발생장소_시도_시군구,발생장소_시도_시군구_읍면동,CTPRVN_CD,SIG_CD,EMD_CD,매핑수준,매핑방법
0,2024,9,29,15:41,일,2024,9,30,16:30,전북,...,산업현장실화,0.31,산동,전북 남원,전북 남원 산동,45,45190,45190410,완전 매칭,"시도,시군구,읍면동"
1,2024,9,10,15:55,화,2024,9,10,18:00,경남,...,성묘객실화(벌집소각),0.10,내이,경남 밀양,경남 밀양 내이,48,48270,48270102,완전 매칭,"시도,시군구,읍면동"
2,2024,9,10,14:35,화,2024,9,10,17:52,충남,...,원인미상,0.03,규암,충남 부여,충남 부여 규암,44,44760,44760310,완전 매칭,"시도,시군구,읍면동"
3,2024,9,10,14:24,화,2024,9,10,22:00,경북,...,조사중,1.00,화동,경북 상주,경북 상주 화동,47,47250,47250400,완전 매칭,"시도,시군구,읍면동"
4,2024,9,5,13:51,목,2024,9,5,16:10,경북,...,농산폐기물소각,0.05,녹전,경북 안동,경북 안동 녹전,47,47170,47170430,완전 매칭,"시도,시군구,읍면동"


In [84]:
# 매핑 결과를 CSV 파일로 저장
fire_df_final.to_csv(os.path.join(path, '산불데이터에코드매핑.csv'), index=False, encoding='cp949')

In [85]:
# 매핑수준 통계
print(fire_df_final['매핑수준'].value_counts())


매핑수준
완전 매칭    1485
수동 매핑      70
세종 매칭      15
Name: count, dtype: int64


In [86]:
# 결측치 개수
fire_df_final[(fire_df_final["매핑수준"] != "세종 매칭") & (fire_df_final["매핑수준"] != "완전 매칭") & (fire_df_final["매핑수준"] != "수동 매핑")].shape[0]

0